  # Main simplificado

In [1]:
# Librerías básicas
import pandas as pd
import numpy as np
from tabulate import tabulate
import math

# Librerías de visualización
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

sns.set(style="whitegrid",font_scale=1, palette="pastel")

#Libreria para separacion de datos train y test
import pickle
from pprint import pprint
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, MinMaxScaler, FunctionTransformer, PowerTransformer
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier

from sklearn.model_selection import cross_validate, cross_val_score
 
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, r2_score,  make_scorer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, MinMaxScaler, FunctionTransformer, PowerTransformer
from sklearn.compose import make_column_transformer, ColumnTransformer


# Librerias a usar para el modelo de machine learning
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_hist_gradient_boosting

In [2]:
# Flag que indica, cuando es True, que es la última vez que se entrena al modelo definitivo y se puede guardar
save_pickle = False

In [3]:
def model_pred(model, X, y, flag):  
   
    # Define a column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                    ('scaler', MinMaxScaler())]), num_cols),
            ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))]), cat_cols)])

    # Create a pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                ('model', model)])

    # Define the metrics for evaluation
    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, average='weighted'),
        'recall': make_scorer(recall_score, average='weighted'),
        'precision': make_scorer(precision_score, average='weighted')
    }

    # Perform 5-fold cross-validation
    cv_results = cross_validate(pipeline, X, y, cv=5, scoring=scoring, return_train_score=True)

    # Calculate overfitting as the difference between training and validation accuracy
    overfitting = np.mean(cv_results['train_accuracy']) - np.mean(cv_results['test_accuracy'])

    # Store the evaluation results
    results = {
        'fit_time': np.mean(cv_results['fit_time']),
        'accuracy': np.mean(cv_results['test_accuracy']),
        'f1': np.mean(cv_results['test_f1']),
        'recall': np.mean(cv_results['test_recall']),
        'precision': np.mean(cv_results['test_precision']),
        'overfitting': overfitting
    }

    # Train the model on the entire dataset
    pipeline.fit(X, y)
    
    if flag:
        # Guardar el pipeline usando Pickle. 
        with open('data_pipeline.pkl', 'wb') as file:
            pickle.dump(pipeline, file)

    return results

In [4]:
#Lectura del dataset
df = pd.read_csv("airline_passenger_satisfaction.csv")

In [5]:
# Configura Pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


In [6]:
df['Customer Type'].head()

0       Loyal Customer
1    disloyal Customer
2       Loyal Customer
3       Loyal Customer
4       Loyal Customer
Name: Customer Type, dtype: object

In [7]:
# Cambiar solo 'disloyal Customer' a 'Disloyal Customer'. Acordarme de ponerlo en el pipeline
df.loc[df['Customer Type'] == 'disloyal Customer', 'Customer Type'] = 'Disloyal Customer'

In [8]:
# Para llenar los valores nulos con la media: (MIRAR SI SE IMPUTAN CON MEDIA O MEDIANA)
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].mean())

In [9]:
# Para convertir una columna de tipo float a int 
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].astype(int)

In [10]:
# Se borran las columnas que no nos hacen falta
columna_a_borrar = ['Unnamed: 0', "id"]
df = df.drop(columna_a_borrar, axis=1, inplace=True)

In [11]:
# Outliers

In [12]:
mean = df["Flight Distance"].mean()
std = df["Flight Distance"].std()

threshold = 3 * std

outliers = df[abs(df["Flight Distance"] - mean) > threshold]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
df = df.drop(index=outliers.index)

In [ ]:
#def model_pred(model, X, y, flag):
    

    #num_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy="mean")),
    #('scaler', MinMaxScaler())
    #])

    #cat_transformer = Pipeline(steps=[
    #    ('imputerc', SimpleImputer(strategy="most_frequent")),    
    #    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)), #Junto con la opción de Javi
    #])

    #Aplicar los transformer a nuestras features usando ColumnTransformer, es nuestro pre-procesamiento.

   # preprocessor = ColumnTransformer(
   #     transformers=[
   #        ('numeric', num_transformer, num_cols),
   #        ('categorical', cat_transformer, cat_cols)
   #     ])
    
    #PIPELINE
    # Estimator o modelo a aplicar
    #pipeline = Pipeline(steps = [
    #    ('preprocessor', preprocessor),
    #    ('modelo', model)
    #])
    
    # Define the metrics for evaluation
    scoring = ['accuracy', 'f1', 'recall', 'precision']
    
    # Initialize a dictionary to store the results
    results = {}
   
    # Definir el objeto StratifiedKFold para la validación cruzada estratificada
    # La varible objetivo está desbalanceada
    #stratified_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    
    # Lo uso en la Validación cruzada
    #cv_results = cross_validate(pipeline, X, y, cv=stratified_cv, 
                                scoring= scoring, return_train_score=True )
    
    #Calculate overfitting as the difference between training and validation accuracy
    #overfitting = np.mean(cv_results['train_accuracy']) - np.mean(cv_results['test_accuracy'])
   
    
    # Store the evaluation results
    #results = {
    #    'fit_time': np.mean(cv_results['fit_time']),
    #    'accuracy': np.mean(cv_results['test_accuracy']),
    #    'f1': np.mean(cv_results['test_f1']),
    #    'recall': np.mean(cv_results['test_recall']),
    #    'precision': np.mean(cv_results['test_precision']),
    #    'overfitting': overfitting
    #}
    
    # Train the model on the entire dataset
    #pipeline.fit(X, y)
    
    # Print the evaluation results for the current model
    #print(f"{model}:\n")
    #print(f"Accuracy:  {results['accuracy']}")
    #print(f"Recall:    {results['recall']}")
    #print(f"Precision: {results['precision']}")
    #print(f"F1:        {results['f1']}")
    #print(f"Overfitting: {results['overfitting']*100:.2f}%")
    #print("="*50)
         
    
    
    #if flag:
    ##    # Guardar el pipeline usando Pickle. Por qué  lo estoy guardando aquí?
    #    with open('data_pipeline.pkl', 'wb') as file:
    #        pickle.dump(pipeline, file)
    
    #return results


In [ ]:
# Separación de características
num_cols = ["Age","Flight Distance","Inflight wifi service","Departure/Arrival time convenient", "Ease of Online booking",
                 "Gate location", "Food and drink", "Online boarding", "Seat comfort", "Inflight entertainment", "On-board service",
                 "Leg room service", "Baggage handling", "Checkin service", "Inflight service", "Cleanliness", "Departure Delay in Minutes",
                 "Arrival Delay in Minutes"]

cat_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

In [ ]:
modelos = [
    LogisticRegression(),
    RandomForestClassifier(),
    #KNeighborsClassifier(),
    # Agrega otros modelos aquí
]

In [ ]:
# Separar Variable Objetivo, target o variable dependiente de las variables independientes
df_f = pd.DataFrame(df)
y = df_f["satisfaction"]
X = df_f.drop(columns="satisfaction")

In [ ]:
# Crea un diccionario para almacenar los resultados de cada modelo
resultados_por_modelo = {}

In [ ]:
# Itera sobre los modelos
for modelo in modelos:
    # Entrena y evalúa el modelo actual
    resultados = model_pred(modelo, X, y)
    
    # Almacena los resultados en el diccionario
    nombre_modelo = type(modelo).__name__
    resultados_por_modelo[nombre_modelo] = resultados

# Imprime los resultados para cada modelo
for nombre_modelo, resultados in resultados_por_modelo.items():
    print(f"Resultados para el modelo {nombre_modelo}:")
    for metrica, valor in resultados.items():
        print(f"{metrica}: {valor}")
    print("=" * 50)

In [ ]:
#model_pred(LogisticRegression(), X, y, save_pickle)

In [ ]:
#model_pred(AdaBoostClassifier(n_estimators=200,  random_state=1), X, y, save_pickle)